## Splitting whole files into 16 bar chunks

In [10]:
import os 

path_output = '/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/txts_drums/'
path_input = '/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/txts_dg-dg-b/'

## Assess there is an equal number of measures in each file

In [11]:
output = []
input = []

for file in os.listdir(path_output):
    output.append(os.path.join(path_output, file))

for file in os.listdir(path_input):
    input.append(os.path.join(path_input, file))

In [12]:
assert len(input) == len(output)

In [5]:
# Function to count occurrences of a string in a file
def count_occurrences_in_file(file_path, search_string):
    count = 0
    try:
        with open(file_path, 'r') as file:
            for line in file:
                count += line.count(search_string)
    except Exception as e:
        print("Could not process file {}: {}".format(file_path, e))
    return count

In [19]:
print(input[100])
print('\n')
print(output[100])

/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/txts_dg-dg-b/Obtest - Auka Seniems Dievams.gp4.tokens.txt


/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/txts_drums/Obtest - Auka Seniems Dievams.gp4.tokens.txt


In [22]:
search_string = 'new_measure'
exceptions = []

for file in input:
    if count_occurrences_in_file(file, search_string) == count_occurrences_in_file(os.path.join(path_output, file.split('/')[-1]), search_string):
        continue
    else:
        print('Here is an exception: {}', file)
        exceptions.append(file)

All the tracks seem to have the same number of bars, as expected.